UNIVERSIDAD AUTONOMA DE CHIHUAHUA

Facultad de Ingeniería

Shakespeare

Alejandro Arturo González Flores

8CC2

Matricula: 348552

Docente: Jesus Roberto López Santillán

Data Science

Chihuahua, Chih. 28 de Noviembre del 2023


In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
shakespeare_url = "https://homl.info/shakespeare"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
  shakespeare_text = f.read()

In [3]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([shakespeare_text])

In [4]:
print(tokenizer.texts_to_sequences(["First"]))
print(tokenizer.sequences_to_texts([[20,6,9,8,3]]))
max_id = len(tokenizer.word_index)
print(max_id)
dataset_size=sum(x for x in tokenizer.word_counts.values())
print(type(dataset_size))
print(dataset_size)

[[20, 6, 9, 8, 3]]
['f i r s t']
39
<class 'int'>
1115394


In [5]:
import numpy as np

In [6]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text]))-1
print(encoded)

[19  5  8 ... 20 26 10]


In [7]:
train_size=dataset_size * 90 //100
dataset=tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [8]:
n_steps = 100
window_length = n_steps + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [9]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [10]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[: ,:-1], windows[:, 1:]))

In [11]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth = max_id), Y_batch)
)
dataset = dataset.prefetch(1)

In [12]:
'''
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None,max_id],dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history=model.fit(dataset, epochs=20)
model.save('my_model.h5')
'''

'\nmodel = keras.models.Sequential([\n    keras.layers.GRU(128, return_sequences=True, input_shape=[None,max_id],dropout=0.2),\n    keras.layers.GRU(128, return_sequences=True,\n                     #dropout=0.2, recurrent_dropout=0.2),\n                     dropout=0.2),\n    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))\n])\nmodel.compile(loss="sparse_categorical_crossentropy", optimizer="adam")\nhistory=model.fit(dataset, epochs=20)\nmodel.save(\'my_model.h5\')\n'

In [13]:
model = tf.keras.models.load_model('my_model.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, None, 128)         64896     
                                                                 
 gru_1 (GRU)                 (None, None, 128)         99072     
                                                                 
 time_distributed (TimeDist  (None, None, 39)          5031      
 ributed)                                                        
                                                                 
Total params: 168999 (660.15 KB)
Trainable params: 168999 (660.15 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
def preprocess(texts):
  X = np.array(tokenizer.texts_to_sequences(texts))-1
  return tf.one_hot(X, max_id)

In [15]:
X_new = preprocess(["How are yo"])
Y_pred_prob = model.predict(X_new)
Y_pred_classes = np.argmax(Y_pred_prob, axis=-1)

predicted_text = tokenizer.sequences_to_texts(Y_pred_classes + 1)[0][-1]

1/1 [==============================] - 3s 3s/step


In [16]:
def next_char(text, temperature=1):
  X_new = preprocess([text])
  y_proba = model.predict(X_new)[0, -1:, :]
  rescaled_logits = tf.math.log(y_proba)/temperature
  char_id = tf.random.categorical(rescaled_logits, num_samples=1) +1
  return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [17]:
def complete_text(text, n_chars=50, temperature=1):
  for _ in range(n_chars):
    text += next_char(text,temperature)
  return text

In [18]:
print(complete_text("t", temperature=0.2))
print(complete_text("w", temperature=1))
print(complete_text("w", temperature=2))

1/1 [==============================] - 0s 99ms/step
t and bear in her.

gremio:
and that i shall be rea
1/1 [==============================] - 0s 27ms/step
with all captista such.
came it, and she may unders
1/1 [==============================] - 0s 48ms/step
wil quettio, yrully
othar: bip,desime,' lont. he th


In [19]:
tf.random.set_seed(42)
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
  dataset= tf.data.Dataset.from_tensor_slices(encoded_part)
  dataset= dataset.window(window_length, shift=n_steps, drop_remainder=True)
  dataset= dataset.flat_map(lambda window: window.batch(window_length))
  datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch)
)
dataset= dataset.prefetch(1)

In [20]:
stateful_model=tf.keras.models.load_model('my_stateful_model.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, None, 128)         64896     
                                                                 
 gru_1 (GRU)                 (None, None, 128)         99072     
                                                                 
 time_distributed (TimeDist  (None, None, 39)          5031      
 ributed)                                                        
                                                                 
Total params: 168999 (660.15 KB)
Trainable params: 168999 (660.15 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
'''
stateful_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=0.2, batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])
'''

'\nstateful_model = keras.models.Sequential([\n    keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=0.2, batch_input_shape=[batch_size, None, max_id]),\n    keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=0.2),\n    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))\n])\n'

In [22]:
class ResetStatesCallback(keras.callbacks.Callback):
  def on_epoch_begin(self, epoch, logs):
    self.model.reset_states()

In [23]:
'''
stateful_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
stateful_model.fit(dataset,epochs=50, callbacks=[ResetStatesCallback()])
stateful_model.save('my_stateful_model.h5')
'''

'\nstateful_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")\nstateful_model.fit(dataset,epochs=50, callbacks=[ResetStatesCallback()])\nstateful_model.save(\'my_stateful_model.h5\')\n'

In [24]:
'''
stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])
'''

'\nstateless_model = keras.models.Sequential([\n    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),\n    keras.layers.GRU(128, return_sequences=True),\n    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))\n])\n'

In [25]:
'''
stateless_model.build(tf.TensorShape([None,None,max_id]))
stateless_model.set_weights(model.get_weights())
stateless_model.save('my_stateless_model.h5')
'''

"\nstateless_model.build(tf.TensorShape([None,None,max_id]))\nstateless_model.set_weights(model.get_weights())\nstateless_model.save('my_stateless_model.h5')\n"

In [26]:
stateless_model=tf.keras.models.load_model('my_stateless_model.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, None, 128)         64896     
                                                                 
 gru_1 (GRU)                 (None, None, 128)         99072     
                                                                 
 time_distributed (TimeDist  (None, None, 39)          5031      
 ributed)                                                        
                                                                 
Total params: 168999 (660.15 KB)
Trainable params: 168999 (660.15 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
tf.random.set_seed(42)
print(complete_text("t"))

1/1 [==============================] - 0s 20ms/step
thing idding hanour.

grumio:
will you go? woy, sir


In [28]:
(X_train,y_train), (X_test, y_test) = keras.datasets.imdb.load_data()
print(X_train[0][:10])

17464789/17464789 [==============================] - 0s 0us/step
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]


In [29]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ +3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>","<sos>","<unk>")):
  id_to_word[id_] = token

" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

1641221/1641221 [==============================] - 0s 0us/step


'<sos> this film was just brilliant casting location scenery story'

In [30]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteA44DMR/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteA44DMR/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteA44DMR/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [31]:
def preprocess(X_batch, y_batch):
  X_batch = tf.strings.substr(X_batch, 0, 300)
  X_batch = tf.strings.regex_replace(X_batch, b"<br\\s*/?>", b" ")
  X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z]", b" ")
  X_batch = tf.strings.split(X_batch)
  return X_batch.to_tensor(default_value=b"<pad>"), y_batch


In [33]:
from collections import Counter
vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
  for review in X_batch:
    vocabulary.update(list(review.numpy()))

In [34]:
print(vocabulary.most_common()[:3])

[(b'<pad>', 224494), (b'the', 61156), (b'a', 38569)]


In [35]:
vocab_size =1000
truncated_vocabulary=[
    word for word, count in vocabulary.most_common()[:vocab_size]
]

In [36]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table= tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [37]:
tbll=table.lookup(tf.constant([b"This movie was faaaantastic".split()]))
print(tbll)

tf.Tensor([[  24   12   13 1771]], shape=(1, 4), dtype=int64)


In [38]:
def encode_words(X_batch, y_batch):
  return table.lookup(X_batch), y_batch

train_set = datasets["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [39]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size, input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 26s 25ms/step - loss: 0.6545 - accuracy: 0.5844
Epoch 2/5
782/782 [==============================] - 13s 17ms/step - loss: 0.4890 - accuracy: 0.7630
Epoch 3/5
782/782 [==============================] - 11s 14ms/step - loss: 0.4304 - accuracy: 0.8010
Epoch 4/5
782/782 [==============================] - 12s 15ms/step - loss: 0.3830 - accuracy: 0.8307
Epoch 5/5
782/782 [==============================] - 10s 12ms/step - loss: 0.3413 - accuracy: 0.8568


In [40]:
model.save('sentiment_analysis_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
